# 🤗🔍 Basic RAG using Hugging Face Embeddings + LLM
This notebook demonstrates a simple Retrieval-Augmented Generation (RAG) pipeline using Hugging Face models for both embeddings and generation.
We'll use LangChain components to wire the system together.

In [ ]:
# Load API keys from `.env` file for Hugging Face authentication
from dotenv import load_dotenv
import os

load_dotenv()

True

In [ ]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline

In [4]:
# Load PDF and split it into chunks
pdf_file = 'sample.pdf'
chunk_size = 1000
chunk_overlap = 200

loader = PyPDFLoader(pdf_file)
documents = loader.load()

# Split the document into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
texts = text_splitter.split_documents(documents)

/Users/chirag/miniconda3/envs/LLMs-env/lib/python3.10/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [5]:
texts[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Macintosh)', 'creationdate': '2024-06-18T14:09:48-07:00', 'moddate': '2024-06-18T14:10:14-07:00', 'trapped': '/False', 'source': 'sample.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Before using iPhone, review the iPhone User Guide  at  \nsupport.apple.com/guide/iphone .\nSafety and Handling\nSee “Safety, handling, and support” in the iPhone  \nUser Guide .\nExposure to Radio Frequency\nOn iPhone, go to Settings > General > Legal &  \nRegulatory > RF Exposure. Or go to apple.com/  \nlegal/rfexposure .\nBattery and Charging\nAn iPhone battery should only be repaired by a trained \ntechnician to avoid battery damage, which could cause \noverheating, fire, or injury. Batteries should be recycled \nor disposed of separately from household waste and \naccording to local environmental laws and guidelines. For \ninformation about Apple lithium-ion batteries and battery \nservice a

In [6]:
len(texts)

8

In [ ]:
# 🔧 Specify the Hugging Face embedding model to use
model_name = "sentence-transformers/all-mpnet-base-v2"

# ⚙️ Model-specific arguments (e.g., run on CPU)
model_kwargs = {'device': 'cpu'}

# 🧮 Embedding behavior settings
# normalize_embeddings=False ensures raw embeddings are used (not L2-normalized)
encode_kwargs = {'normalize_embeddings': False}

# 🤗 Initialize the HuggingFaceEmbeddings class with specified settings
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Create FAISS vector store from the embedded chunks
vectorstore = FAISS.from_documents(
    texts,
    embedding=hf
)

In [9]:
retriever = vectorstore.as_retriever()

In [ ]:
# Define the custom prompt template for generation

template = """
You are a customer service agent for a apple mobile company. 
You have been given the following information about the customer query and the context.
Customer Query: {query}
Context: {context}

### Answer: 
The answer should be based on the context provided.
Your task is to answer the customer query based on the context provided. If the question is not related to the context, please say "I don't know or Do Not Answer it just say please ask me question related to Apple Mobiles only".
Do not make up any information or provide any personal opinions or experiences.
Please answer in a friendly and professional manner.
"""

In [ ]:
# Wrap prompt inside ChatPromptTemplate for compatibility with chat models
prompt = ChatPromptTemplate.from_messages(
    [
	("system", "You are a helpful assistant."),
	("human", template),
    ]
)

print(prompt)

input_variables=['context', 'query'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template='\nYou are a customer service agent for a apple mobile company. \nYou have been given the following information about the customer query and the context.\nCustomer Query: {query}\nContext: {context}\n\nAnswer: \nThe answer should be based on the context provided.\nYour task is to answer the customer query based on the context provided. If the question is not related to the context, please say "I don\'t know or Do Not Answer it just say please ask me question related to Apple Mobiles only".\nDo not make up any information or provide any personal opinions or experiences.\nPlease answer in a friendly and p

In [ ]:
# Authenticate with Hugging Face Hub
from huggingface_hub import login
login(os.getenv("HUGGINGFACE_TOKEN"))

In [ ]:
# 🚀 Load a Hugging Face LLM for text generation using Transformers and wrap it with LangChain

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 🧠 Model ID from Hugging Face Hub (you can replace this with any causal LLM)
model_id = "google/gemma-2b-it"

# 🗝️ Load the tokenizer for the chosen model (handles input formatting and tokenization)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 🧱 Load the actual causal language model (Gemma-2B-IT in this case)
model = AutoModelForCausalLM.from_pretrained(model_id)

# 🛠️ Create a text-generation pipeline from the loaded model and tokenizer
pipe = pipeline(
    "text-generation",        # task type
    model=model,              # pre-trained model
    tokenizer=tokenizer,      # matching tokenizer
    max_new_tokens=512,       # output token limit
    do_sample=True,           # enables sampling (vs deterministic greedy decoding)
    temperature=0.7,          # sampling randomness (lower = more focused)
    top_p=0.95,               # nucleus sampling cutoff
    repetition_penalty=1.1    # discourages repeating phrases
)

# 🔁 Wrap the Hugging Face pipeline in LangChain's HuggingFacePipeline for integration
llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
# Combine content of all retrieved documents into a single string
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# Custom parser to extract clean answer from model output
class CleanAnswerParser(StrOutputParser):
    def parse(self, text: str) -> str:
        # Only keep the part after '### Answer:'
        if "### Answer:" in text:
            return text.split("### Answer:")[-1].strip().split("\n")[0]
        return text.strip()

In [ ]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "query": RunnablePassthrough()}
    | prompt
    | llm
    
    | CleanAnswerParser()
)

In [ ]:
# Example 1: Ask a domain-specific question
response = rag_chain.invoke('Is there a warranty on the phone?')
print('📦 Answer:', response)

📦 Answer: Hello! Yes, there is a warranty on your phone. Apple provides a one-year limited warranty for the included hardware product and accessories against defects in materials and workmanship from the date of original retail purchase. Please note that the warranty does not cover normal wear and tear, or damage caused by accident or abuse.



In [ ]:
# Example 2: Ask a general knowledge question
response = rag_chain.invoke('What is GTA 6?')
print('🎮 Answer:', response)

🎮 Answer: I don't know.


<!-- Font Awesome CDN (Add in <head> if not already included) -->
<link
  rel="stylesheet" 
  href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.0/css/all.min.css"
/>

<!-- Social Footer Section -->
<div style="
  background-color:rgb(199, 195, 195);
  padding: 40px 30px;
  border-radius: 20px;
  box-shadow: 0 4px 12px rgba(0,0,0,0.08);
  font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  font-size: 18px;
  max-width: 900px;
  margin: 60px auto 30px;
  text-align: center;
  color: #444;
">
<!-- End of Notebook Note -->
  <h2 style="margin-bottom: 10px;">📘 End of Notebook</h2>
  <p style="color: #666; font-size: 14px;">
    Thank you for exploring! Feel free to connect via the links below.
  </p>

  <!-- Social Icons -->
<div style="
  display: flex;
  gap: 25px;
  align-items: center;
  flex-wrap: wrap;
  justify-content: center;
  margin-bottom: 25px;
">
  <!-- LinkedIn -->
  <a href="https://www.linkedin.com/in/ChiragB254" target="_blank" style="text-decoration: none; color: #0077b5;">
    <i class="fab fa-linkedin fa-lg"></i> LinkedIn
  </a>

  <!-- GitHub -->
  <a href="https://github.com/ChiragB254" target="_blank" style="text-decoration: none; color: #333;">
    <i class="fab fa-github fa-lg"></i> GitHub
  </a>

  <!-- Instagram -->
  <a href="https://www.instagram.com/data.scientist_chirag" target="_blank" style="text-decoration: none; color: #E1306C;">
    <i class="fab fa-instagram fa-lg"></i> Instagram
  </a>

  <!-- Email -->
  <a href="mailto:devchirag27@gmail.com" style="text-decoration: none; color: #D44638;">
    <i class="fas fa-envelope fa-lg"></i> Email
  </a>

  <!-- X (Twitter) -->
  <a href="https://x.com/ChiragB254" target="_blank" style="text-decoration: none; color: #000;">
    <i class="fab fa-x-twitter fa-lg"></i> X.com
  </a>
  </div>

  <p style="font-size: 13px; color: black; font-style: italic; margin-top: 8px;">
    <strong>Made with ❤️ by Chirag Bansal</strong>
  </p>
</div>

